# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI
import nest_asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

# Aplicar nest_asyncio para Jupyter
nest_asyncio.apply()

# Inicializar Clientes (Sync para utilidades, Async para Agentes)
client = OpenAI(api_key=api_key)
aclient = AsyncOpenAI(api_key=api_key)
print("✅ Clientes OpenAI inicializados.")

## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [ ]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [ ]:
# 2. INGESTIÓN DE DATOS (Optimizado con Caché ⚡)
import os
import json
from pathlib import Path
from openai import OpenAI

# Configuración
CACHE_FILE = Path("vector_store_cache.json")
PDF_PATH = Path("../data/raw/TENERIFE.pdf")

# Comprobación del PDF (Path Correction)
if not PDF_PATH.exists():
    if Path("data/raw/TENERIFE.pdf").exists():
        PDF_PATH = Path("data/raw/TENERIFE.pdf")
    else:
        raise FileNotFoundError(f"No existe el PDF en {PDF_PATH}. Ajusta ruta.")

print("PDF:", PDF_PATH)

# 1. Detectar gestor de Vector Stores
vs_manager = getattr(client, "vector_stores", None) or getattr(getattr(client, "beta", None), "vector_stores", None)
if vs_manager is None:
    raise RuntimeError("Tu versión del SDK no expone 'vector_stores'.")

# 2. Intentar recuperar de Caché
vector_store_id = None
if CACHE_FILE.exists():
    try:
        with open(CACHE_FILE, "r") as f:
            data = json.load(f)
            cached_id = data.get("vector_store_id")
            # Verificar si sigue vivo en OpenAI
            vs_manager.retrieve(cached_id)
            vector_store_id = cached_id
            print(f"[CACHÉ] ⚡ ID recuperado y verificado: {vector_store_id}")
    except Exception as e:
        print(f"[CACHÉ] ⚠️ ID inválido o expirado ({e}). Se creará uno nuevo.")

# 3. Crear e Ingestar (Solo si no hay caché)
if not vector_store_id:
    print("[INGESTA] 🐢 Iniciando proceso de subida (esto tomará un momento)...")
    vector_store = vs_manager.create(
        name="Tenerife_VS_Final",
        chunking_strategy={"type": "static", "static": {"max_chunk_size_tokens": 1000, "chunk_overlap_tokens": 200}},
    )
    vector_store_id = vector_store.id
    
    with open(PDF_PATH, "rb") as f:
        vs_manager.file_batches.upload_and_poll(
            vector_store_id=vector_store_id,
            files=[f],
        )
    
    # Guardar nuevo caché
    with open(CACHE_FILE, "w") as f:
        json.dump({"vector_store_id": vector_store_id}, f)
    print(f"[INGESTA] 💾 Completado y guardado en caché: {vector_store_id}")

print(f"✅ Fase 2 Lista. ID Activo: {vector_store_id}")


## 4. Fase 3: Configuración del Agente RAG 🤖
Configuramos el agente con capacidad de usar el Vector Store. 
**Nota**: Incluimos la instalación de librerías en el kernel para asegurar que el entorno de ejecución tenga todo lo necesario.

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from agents import Agent, Runner, FileSearchTool

# Usamos el ID obtenido en Fase 2
current_vs_id = vector_store_id
print(f"🔗 Conectando agente al Vector Store: {current_vs_id}")

# 1. Definir la herramienta de búsqueda de archivos
file_search_tool = FileSearchTool(vector_store_ids=[current_vs_id])

# 2. Instrucciones del Agente (System Prompt)
INSTRUCTIONS = """
Eres un experto guía turístico de Tenerife.
Usa la herramienta file_search para encontrar información en la guía PDF adjunta.
Si la información no está en el PDF, dilo claramente. No inventes respuestas.
Responde siempre en español, con un tono amable y entusiasta.
"""

# 3. Crear el Agente
agent = Agent(
    name="TenerifeGuide",
    model="gpt-4o-mini",
    instructions=INSTRUCTIONS,
    tools=[file_search_tool]
)

print(f"🤖 Agente '{agent.name}' configurado satisfactoriamente.")

### Prueba Rápida del Agente
Lanzamos una pregunta de prueba para verificar que el RAG funciona.

In [ ]:
async def demo_agente():
    pregunta = "¿Qué actividades puedo hacer en el Teide?"
    print(f"👤 Usuario: {pregunta}")
    
    # Ejecutamos el agente
    resultado = await Runner.run(agent, pregunta)
    print(f"🤖 Agente: {resultado.final_output}")

await demo_agente()

# 4. FASE 4: GENERACIÓN DE DATASET DE EVALUACIÓN

Para medir la calidad de nuestro RAG, necesitamos un "Gold Standard" o dataset de prueba.
En lugar de escribirlo a mano, usaremos un **Agente Auxiliar ('DatasetBuilder')**.

Este agente leerá el PDF y generará pares de Pregunta/Respuesta con referencias exactas.
Esto asegura que las pruebas estén alineadas con el contenido real del documento.

In [ ]:
from typing import List
from pydantic import BaseModel
import json

# Modelo de datos para cada item de evaluación
class EvalItem(BaseModel):
    question: str
    answer: str
    references: List[str]

NUM_EXAMPLES = 5  # Número de pares a generar (puedes aumentar esto luego)
MODEL_GENERATOR = "gpt-4o-mini" # Usamos el mismo modelo capaz

# Definición del Agente Generador
dataset_agent = Agent(
    name="DatasetBuilder",
    model=MODEL_GENERATOR,
    instructions=(
        "Eres un generador de datasets de QA (Pregunta/Respuesta) basado en un PDF accesible vía file_search. "
        "Crea preguntas variadas (literales, síntesis, curiosidades) sobre Tenerife y respuestas concisas apoyadas en el texto. "
        "Debes devolver un JSON con una lista de objetos que sigan la estructura {question, answer, references}. "
        "Las 'references' deben ser citas textuales exactas del PDF que soporten la respuesta. NO inventes texto."
    ),
    tools=[file_search_tool], # Reutilizamos la tool configurada en Fase 3
    output_type=List[EvalItem],
)

print("✅ Agente DatasetBuilder configurado.")

In [ ]:
# Prompt para disparar la generación
dataset_prompt = (
    f"Genera {NUM_EXAMPLES} pares de pregunta/respuesta basados únicamente en el PDF de Tenerife. "
    "Incluye las referencias textuales relevantes en el campo 'references'. "
    "Responde solo con un JSON válido."
)

print(f"⏳ Generando {NUM_EXAMPLES} ejemplos de evaluación... (Esto puede tardar unos segundos)")

try:
    # Ejecución del agente
    dataset_result = await Runner.run(dataset_agent, dataset_prompt)
    
    # Extracción del resultado tipado
    eval_dataset = dataset_result.final_output_as(List[EvalItem])
    
    print(f"\n✅ Generados {len(eval_dataset)} ejemplos:")
    for i, item in enumerate(eval_dataset, 1):
        print(f"[{i}] P: {item.question}")
        print(f"    R: {item.answer[:100]}...")

except Exception as e:
    print(f"❌ Error generando dataset: {e}")


In [ ]:
# Guardar el dataset en disco para no regenerarlo siempre
DATASET_PATH = Path("eval_dataset_tenerife.json")

if 'eval_dataset' in locals() and eval_dataset:
    with open(DATASET_PATH, "w", encoding="utf-8") as f:
        json.dump([item.model_dump() for item in eval_dataset], f, ensure_ascii=False, indent=2)
    print(f"💾 Dataset guardado en: {DATASET_PATH.absolute()}")
else:
    print("⚠️ No hay dataset para guardar.")

# 5. FASE 5: VERIFICACIÓN Y MÉTRICAS (JUEZ LLM)

Ya tenemos Agente y Dataset. Ahora necesitamos saber **qué tan bien** funciona.
Usaremos un segundo LLM (Juez) para calificar cada respuesta del Agente según:
1.  **Faithfulness (Fidelidad):** ¿Se inventó algo?
2.  **Answer Correctness:** ¿Coincide con la respuesta esperada?
3.  **Context Precision:** ¿Los trozos de PDF recuperados eran útiles?

In [ ]:
import asyncio
import statistics
from typing import Dict, Any, List
from pydantic import BaseModel

# Aseguramos imports
from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Definición de la Salida del Juez
class JudgeScores(BaseModel):
    faithfulness: float
    answer_correctness: float
    context_precision: float
    reasoning: str

# Definición del Juez (usamos gpt-4o-mini por ser rápido y capaz)
judge_agent = Agent(
    name="EvalJudge",
    model="gpt-4o-mini",
    instructions=(
        "Actúas como un juez evaluador de sistemas RAG. "
        "Debes generar puntuaciones (0.0 a 1.0) para: "
        "1. Faithfulness: ¿La respuesta se basa solo en el contexto recuperado? (1=Sí, 0=Alucinación total). "
        "2. Answer Correctness: ¿La respuesta del agente coincide semánticamente con la respuesta de referencia? "
        "3. Context Precision: ¿Los contextos recuperados son relevantes para la pregunta? "
        "Devuelve un JSON con los campos numéricos y un breve 'reasoning'."
    ),
    output_type=JudgeScores
)

print("✅ Agente Juez configurado.")

In [ ]:
# Función auxiliar para extraer el texto de los contextos recuperados
def extract_contexts(run_result) -> List[str]:
    contexts = []
    # Inspeccionamos items generados buscando llamadas a tools
    if hasattr(run_result, 'new_items'):
        for item in run_result.new_items:
            raw = getattr(item, "raw_item", None)
            # Detectar llamada a file_search
            if getattr(raw, "type", None) == "file_search_call":
                for res in getattr(raw, "results", []) or []:
                    text = getattr(res, "text", None) or getattr(res, "content", None)
                    if text: contexts.append(text)
    return contexts

# Función que evalúa UNA sola pregunta
async def evaluate_example(agent_to_test: Agent, example: EvalItem):
    # 1. El Agente a testear responde la pregunta
    run_res = await Runner.run(agent_to_test, example.question)
    agent_answer = run_res.final_output
    contexts = extract_contexts(run_res)
    
    # 2. El Juez evalúa la calidad
    context_text = "\n---\n".join(contexts[:3]) # Top 3 contextos
    judge_prompt = (
        f"Pregunta: {example.question}\n"
        f"Respuesta Agente: {agent_answer}\n"
        f"Respuesta Referencia: {example.answer}\n"
        f"Contextos Recuperados: {context_text}\n"
        "Evalúa con rigor."
    )
    
    judge_res = await Runner.run(judge_agent, judge_prompt)
    scores = judge_res.final_output_as(JudgeScores)
    
    return {
        "question": example.question,
        "agent_answer": agent_answer,
        "reference": example.answer,
        "faithfulness": scores.faithfulness,
        "correctness": scores.answer_correctness,
        "ctx_precision": scores.context_precision,
        "reasoning": scores.reasoning
    }

In [ ]:
# Ejecución Masiva de la Evaluación
print(f"⏳ Evaluando los {len(eval_dataset)} ejemplos con el Juez... Estiman 10-20 segundos.")

results = []
for ex in eval_dataset:
    res = await evaluate_example(agent, ex)
    results.append(res)
    # Pequeño print de progreso
    print(f". Completado: {ex.question[:30]}... -> F:{res['faithfulness']} C:{res['correctness']}")

print("\n✅ Evaluación Finalizada.")

In [ ]:
# Reporte de Resultados
try:
    import pandas as pd
    df_results = pd.DataFrame(results)
    print("\nPromedios Generales:")
    df_results.to_csv("eval_results.csv", index=False, encoding='utf-8')
    print("💾 Resultados guardados en 'eval_results.csv'")
    print(df_results[["faithfulness", "correctness", "ctx_precision"]].mean())
    display(df_results) # Pretty print en notebook
except ImportError:
    # Fallback si no hay pandas
    print("\nResultados Detallados:")
    for r in results:
        print(r)

# 6. FASE 6: FUNCTION CALLING (CLIMA)

Implementamos una herramienta externa (`get_weather`) usando el decorador `@function_tool`.
Esto permite al Agente invocar código Python real (o simulado) para obtener estructuras JSON.

In [ ]:
import random
import datetime
import json
from agents import function_tool  # Importante: Decorador para herramientas

# 1. Definimos la función decorada
@function_tool
def get_weather(location: str, date: str = None) -> str:
    """
    Obtiene la previsión del tiempo para una ubicación y fecha dadas.
    
    Args:
        location: Ciudad o lugar (ej: 'Tenerife').
        date: Fecha en formato YYYY-MM-DD o 'hoy'.
    """
    # Log solicitado por requisitos
    print(f"[TOOL LOG] Llamada a get_weather(location='{location}', date='{date}')")
    
    # Mock (Simulación)
    if not date:
        date = datetime.date.today().isoformat()
    
    climates = ["Soleado", "Parcialmente nublado", "Viento moderado"]
    temp = random.randint(20, 28)
    
    return json.dumps({
        "location": location,
        "date": date,
        "condition": random.choice(climates),
        "temperature": f"{temp}ºC",
        "note": "Simulación académica"
    })

# 2. Creamos el Agente con la herramienta correctamente configurada
weather_agent = Agent(
    name="TenerifeGuidePro",
    model="gpt-4o-mini",
    instructions=(
        "Eres el experto turístico definitivo de Tenerife. "
        "Usa file_search para responder dudas sobre lugares basándote en el PDF. "
        "Usa get_weather si el usuario pregunta por el tiempo o clima actual. "
        "Combina ambas fuentes para dar consejos complejos."
    ),
    tools=[file_search_tool, get_weather]
)

print("✅ Agente 'TenerifeGuidePro' configurado correctamente.")

In [ ]:
# Prueba de Function Calling + RAG
pregunta_clima = "¿Qué tiempo hace hoy en el Teide y qué me recomiendas hacer allí según el PDF?"

print(f"👤 Usuario: {pregunta_clima}")
print("⏳ Pensando...")

try:
    resultado_completo = await Runner.run(weather_agent, pregunta_clima)
    print(f"🤖 Agente: {resultado_completo.final_output}")
except Exception as e:
    print(f"❌ Error: {e}")

# 7. FASE 7: ANÁLISIS FINAL Y CONCLUSIONES 📊

En esta última fase, nuestro objetivo es sintetizar todo lo aprendido. No basta con ver números en una tabla; queremos que el propio LLM (actuando como Analista) revise los resultados de la evaluación (Fase 5) y nos dé un informe ejecutivo.

Esto cierra el ciclo: **Ingesta -> RAG -> Evaluación -> Mejora Continua**.

In [ ]:
async def generar_informe_final():
    global df_results
    print("📊 Generando Análisis Final del Proyecto...")
    
    # 1. Recuperación de Datos (Memoria o Disco)
    stats_md = ""
    try:
        if 'df_results' not in globals():
            import pandas as pd
            if os.path.exists("eval_results.csv"):
                print("[INFO] Cargando resultados desde archivo CSV...")
                df_results = pd.read_csv("eval_results.csv")
            else:
                raise FileNotFoundError("No se encontraron resultados en memoria ni en 'eval_results.csv'. Ejecuta la Fase 5.")
        
        # Calcular estadísticas
        stats = df_results[["faithfulness", "correctness", "ctx_precision"]].describe()
        stats_md = stats.to_markdown()
    except Exception as e:
        print(f"⚠️ Error obteniendo métricas: {e}")
        return
    
    # 2. Agente Analista
    analyst_agent = Agent(
        name="ProjectAnalyst",
        model="gpt-4o-mini",
        instructions=(
            "Eres un Analista Senior. Analiza las métricas de evaluación RAG proporcionadas. "
            "Redacta un informe ejecutivo breve (máx 5 líneas) con: "
            "1. Resumen de rendimiento (¿Es fiable?). "
            "2. Veredicto final. "
            "Sé directo y profesional."
        )
    )
    
    prompt_analisis = (
        f"Analiza estas métricas:\n{stats_md}\n\n"
        "Genera el informe final ahora."
    )
    
    informe = await Runner.run(analyst_agent, prompt_analisis)
    print("\n" + "="*40)
    print("📑 INFORME FINAL DEL PROYECTO")
    print("="*40 + "\n")
    print(informe.final_output)

# Ejecutar análisis
await generar_informe_final()

## Fin de la Demostración
¡Felicidades! Has completado el ciclo de vida completo de una aplicación LLM profesional.